# Creating A Facial Detection and Recognition Model
# Idris Nemsia
More documentation will be included in the future version
## References
To build my model, I have consulted some references to guide my coding process.
[1] : J. Loy. Implementing a Facial Recognition System with Neural Networks. Neural Network Projects with Python. Packt Publishing, February 2019.
[2] : Koch et al. Siamese Neural Networks for One-shot Image Recognition. ICML deep learning workshop, vol. 2. 2015.
[3] : Build a Facial Recognition App // Deep Learning Project // Paper2Code Series. https://www.youtube.com/playlist?list=PLgNJO2hghbmhHuhURAGbe6KWpiYZt0AMH

## Initial Preparations Section
These section includes the installation of some of our needed modules. It also imports the modules we will be using.

In [87]:
# Installing Tensorflow and opencv
# !pip install --user tensorflow==2.10.1 opencv-python
#!pip install keras

In [88]:
# Imports
import cv2
import os
import numpy as np
import matplotlib
# Import Sequential and Functional Tenserflow API, as well as layers
import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Input


## Loading our data
We will be using ORL AT&T Dataset of Faces. (*More information will be included in future documentation)
In this section, we initialize our preset input and output arrays (X_train, Y_train, X_test, Y_test).
We import the folder containing the dataset of images. The folder contains 40 folders, each containing pictures of one person. We split the folders into 35 to 5 folders for training and testing data. We convert these images into numpy arrays containing numerical values. We add the numpy arrays of images to our input and output arrays accordingly.

In [100]:
# Initialize empty X,Y training and test arrays
X_train, Y_train = [], []
X_test, Y_test = [], []
# Define the dataset path
dataset_path = "dataset"
# List of DirEntry Objects. These objects contain path attributes.
dataset = os.listdir(dataset_path)
dataset = sorted(dataset)
# Our training data is 1-35, test data is 35-40
# Loop through the folders inside the dataset
for folder_index in range(1,41):
    person_name = dataset[folder_index]
    person_folder_path = dataset_path + "/" + person_name
    person_folder = os.listdir(person_folder_path)
    # Loop through images in each person's folder
    for img_name in person_folder:
        img_path = person_folder_path + '/' + img_name
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img_array = np.array(img)
        img_array = np.expand_dims(img, axis=2)
        # Splitting dataset_path
        if folder_index <= 35:
            X, Y = X_train, Y_train
        else:
            X, Y = X_test, Y_test
        X.append(img_array)
        Y.append(person_name)

img_shape = X_train[0].shape
print(img_shape)
img_width, img_height = img_shape[0], img_shape[1]



(112, 92, 1)


## Creating our Model
### Creating the feature generating model
The layers of this section follow the layers explained in [2]. However, more modifications will be made to this code as Max Pooling layers are incomplete. This is causing the loss value to be NaN during the training of my model.

In [101]:
# The sequential model allows us to connect layers that each receive inputs and can send their outputs to the next layer
feature_generation_network = Sequential()
feature_generation_network.add(Conv2D(filters=64, kernel_size=(10,10), activation='relu',input_shape=img_shape))
feature_generation_network.add(MaxPooling2D())
feature_generation_network.add(Conv2D(filters=128, kernel_size=(7,7), activation='relu'))
feature_generation_network.add(MaxPooling2D())
feature_generation_network.add(Conv2D(filters=128, kernel_size=(4,4), activation='relu'))
feature_generation_network.add(MaxPooling2D())
feature_generation_network.add(Conv2D(filters=256, kernel_size=(4,4), activation='relu'))
feature_generation_network.add(Flatten())
feature_generation_network.add(Dense(units=4096, activation='sigmoid'))

In [102]:
feature_generation_network.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_49 (Conv2D)          (None, 103, 83, 64)       6464      
                                                                 
 max_pooling2d_37 (MaxPoolin  (None, 51, 41, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_50 (Conv2D)          (None, 45, 35, 128)       401536    
                                                                 
 max_pooling2d_38 (MaxPoolin  (None, 22, 17, 128)      0         
 g2D)                                                            
                                                                 
 conv2d_51 (Conv2D)          (None, 19, 14, 128)       262272    
                                                                 
 max_pooling2d_39 (MaxPoolin  (None, 9, 7, 128)      

### Defining the inputs and outputs
The outputs here refer to the network used to generate features inside our siamese neural network for each input. The two outputs of the feature_generation_network from input1 and input2 will then be compared by calculating the Euclidean distance between them.

In [103]:
input1, input2 = Input(shape=img_shape), Input(shape=img_shape)
# Share the single output layer for both inputs
output1, output2 = feature_generation_network(input1), feature_generation_network(input2)

## Defining the output of the siamese_network: Distance between our two inital outputs

In [104]:
from sklearn.metrics.pairwise import euclidean_distances
distance = tf.norm(output1 - output2, axis=1, keepdims=True)

### Initialize Siamese Neural Network Model with inputs and outputs

In [105]:
siamese_network = Model((input1, input2), distance)

### Get Summary of our model

In [106]:
siamese_network.summary()

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_17 (InputLayer)          [(None, 112, 92, 1)  0           []                               
                                ]                                                                 
                                                                                                  
 input_18 (InputLayer)          [(None, 112, 92, 1)  0           []                               
                                ]                                                                 
                                                                                                  
 sequential_14 (Sequential)     (None, 4096)         26364736    ['input_17[0][0]',               
                                                                  'input_18[0][0]']         

## Training our model

### Define Loss Function: Contrastive Loss function

In [107]:
# This follows the loss function explained in [1], [2], [3]
def contrastive_loss(true_pair_label, distance):
    true_pair_label = tf.cast(true_pair_label, tf.float32)
    loss = true_pair_label * tf.square(distance) + (1 - true_pair_label) * tf.square(tf.maximum(1 - distance, 0))
    return loss

### Compile our model

In [108]:
siamese_network.compile(loss=contrastive_loss, optimizer='adam')

### Creating X_train, Y_train, X_test, Y_test using pairs of images and their outputs (0 or 1)

In [109]:
import random
# This function finds a positive pair of images within a set of images
def positive_pair(X, number_of_people):
    picture1_index = random.randint(0,9)
    picture2_index = picture1_index
    while picture2_index == picture1_index:
        picture2_index = random.randint(0,9)
    random_person_index = random.randint(0,number_of_people - 1)
    picture1_index = random_person_index * 10 + picture1_index
    picture2_index = random_person_index * 10 + picture2_index
    return np.array((X[picture1_index], X[picture2_index]))

# This function finds a negative pair of images within a set of images
def negative_pair(X, number_of_people):
    person1_index = random.randint(0,number_of_people - 1)
    person2_index = person1_index
    while person2_index == person1_index:
        person2_index = random.randint(0,number_of_people - 1)
    picture1_index = random.randint(0,9)
    picture2_index = random.randint(0,9)
    picture1_index = person1_index * 10 + picture1_index
    picture2_index = person2_index * 10 + picture2_index
    return np.array((X[picture1_index], X[picture2_index]))

# This function creates a list of random input pairs (X) and a list of their labels (Y) from a set of images.
# It alternates between adding a positive and negative pair of images to X with matching labels to Y.
def create_X_pairs(X, number_of_people, size):
    X_pairs, X_pairs_labels = [], []
    for i in range(size):
        if i % 2 == 0:
            X_pairs.append(positive_pair(X, number_of_people))
            X_pairs_labels.append(1)
        else:
            X_pairs.append(negative_pair(X,number_of_people))
            X_pairs_labels.append(0)
    return (X_pairs, X_pairs_labels)

# Creating our training and data sets for pairs of images
# These are the sets that the siamese network will take the input from.
## Currently, I am still modifying the sizes of my sets of pairs and labels for training
## and testing. I believe that this may be affecting the error of the loss function
## during the training of the model.
X_pairs_train, Y_pairs_train = create_X_pairs(X_train, 35, 100)
X_pairs_test, Y_pairs_test = create_X_pairs(X_test, 5, 5)
# Turning our training and testing sets into numpy arrays
X_pairs_train = np.array(X_pairs_train)
Y_pairs_train = np.array(Y_pairs_train)
X_pairs_test = np.array(X_pairs_test)
Y_pairs_test = np.array(Y_pairs_test)

# Exporting our model to be integrated into a web application

In [ ]:
# Creating numpy arrays for the first and second element of each pair
X1_train = np.array([pair[0] for pair in X_pairs_train])
X2_train = np.array([pair[1] for pair in X_pairs_train])
# Training the model
## This currently runs. However, the loss output is incorrect, as it starts with numerical values but then
## turns into NaN. This negatively affects the training of my siamese network. I am currently working
## on fixing this error.
## I will also be modifying the batch_size and epochs size to see if it affects my results
siamese_network.fit(x=[X1_train, X2_train], y = Y_pairs_train, batch_size = 3, epochs=5)

Epoch 1/5
34/34 [==============================] - 13s 382ms/step - loss: nan
Epoch 2/5
 5/34 [===>..........................] - ETA: 11s - loss: nan

# Next Steps
1) Fixing the current error in the training of my module. (Incorrect loss value output)
2) Testing my module
3) Exporting the module and creating the interface of the application in which I will be using it to record attendance with facial recognition.